# Topic Modeling with Gensim

A **topic model** is an abstraction of the major topics contained in a corpus of texts. "Topic" in this context simply means a pattern of co-occurring words. The assumption is that if there are clearly identified patterns of co-occurring words, those patterns of co-occurring words reveal a latent structure in the corpus of texts. In short, a topic model is a representation of the major themes or structures of a corpus of texts.

`Gensim` is a popular Python library for building topic models. In this notebook we will use `Gensim` to build a topic model of Gibbon's _Decline and Fall of the Roman Empire_. After building a topic model, we will then use `pyLDAvis` to visualize the model so we can evaluate its usefulness.

I highly recommend that you read through `Gensim`'s [documentation](https://radimrehurek.com/gensim/auto_examples/index.html#core-tutorials-new-users-start-here). Much of the code below is adapted from that source.

## Set up

**NOTE**: one of the Python libraries we are using (`pyLDAvis`) can cause problems. Be sure to do the installations in the order that you see them below.

In [1]:
! pip install funcy

In [2]:
! pip install tzdata

In [3]:
! pip install --no-dependencies pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.0 MB/s eta 0:00:0000:0100:01


In [4]:
! pip install wget

In [11]:
!pip install scikit-learn

In [12]:
!pip install gensim

In [13]:
from collections import defaultdict
import wget
from gensim import corpora, models
# from google.colab import files  # <-- only necessary if you are using Colab
# import io # <-- only necessary if you are using Colab
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Upload data

### Class example

In [14]:
url = 'https://raw.githubusercontent.com/msaxton/nlp-data/main/gibbon_sections.csv'
file_name = wget.download(url)
df = pd.read_csv(file_name)
df.head()

 25% [.............                                       ]  4308992 / 16911292

 86% [.............................................       ] 14671872 / 16911292

100% [....................................................] 16911292 / 16911292

,Unnamed: 0,doc_id,text,lemmas
0,0,01-0,The extent and military force of the Roman emp...,extent force empire century comprehend part ea...
1,1,01-1,"exalted situation, had much less to hope than ...",exalt situation hope fear chance arm prosecuti...
2,2,01-2,"and towards the south, the sandy deserts of A...",south desert imitate successor repose mankind ...
3,3,01-3,the love of freedom without the spirit of unio...,love freedom spirit union take arm fierceness ...
4,4,01-4,"line of military stations, which was afterwar...",line station fortify reign rampart erect found...


### Upload your own data sets

If you are using Google Colab:

In [ ]:
# uploaded = files.upload()

In [ ]:
# file_name = # <-- for example: 'data.csv'
# df = pd.read_csv(io.BytesIO(uploaded[file_name]))

If you are using Jupyter Notebooks:

In [21]:
# path = "/"
file_name = "gold_rush1870-1945.csv"
full_path = file_name
df = pd.read_csv(full_path)

## Prepare data for topic model
The Python library we are going to use to make our topic model requires the data to be in a form of a list. Within that list, each "document" is also a list. So it looks something like this:

`[
  ['This is document 1'],
  ['This is document 2'],
  ['This is document 3']
]`

In [22]:
# extract the data out of the DataFrame
documents = df['lemmas'].to_list()

`Gensim` needs each document to be tokenized. We can use [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) to quickly achieve this result. When complete, our data will now look like this:

`[
  ['This', 'is', 'document', '1'],
  ['This', 'is', 'document', '2'],
  ['This', 'is', 'document', '3'],
]`

In [23]:
# tokenize - the syntax below will create a list of lists
texts =[
    [word for word in document.lower().split()]
    for document in documents
]

It takes a lot of preparation to build a useful topic model. An important part of that preparation is to eliminate "noise" from you model. One way to do this is to remove pieces of data that are irrelevant. Here we will remove tokens that only occur once. **You may want to adjust this as you refine your topic model.**

In [24]:
# create a count of each token
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [25]:
# remove words that appear only 1 time
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

## Build topic model

`Gensim` is built around [four core concepts](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#core-concepts):
- **document**: an individual text. In our case, this is an individual section from Gibbon.
- **corpus**: a collection of documents. In our case, this is all the sections from Gibbon put together.
- **vector**: a mathematically convenient representation of a document. Basically, each word in the document is given a numerical id. This allows `Gensim` to do faster calculations behind the scene.
- **model**: an algorithm for transforming vectors from one representation to another. In our case, this will be the LDA model we build.


### Basic topic model



In [26]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
dictionary = corpora.Dictionary(texts)

In [27]:
# create a corpus based off our dictionary and our texts
corpus = [dictionary.doc2bow(text) for text in texts]

In [28]:
# build LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)

In [29]:
# explore topics
lda_model.print_topics()

[(0,
  '0.025*"society" + 0.019*"california" + 0.016*"man" + 0.015*"pioneers" + 0.012*"early" + 0.009*"day" + 0.009*"old" + 0.009*"new" + 0.009*"member" + 0.009*"time"'),
 (1,
  '0.000*"t" + 0.000*"day" + 0.000*"man" + 0.000*"california" + 0.000*"new" + 0.000*"j" + 0.000*"time" + 0.000*"year" + 0.000*"l" + 0.000*"society"'),
 (2,
  '0.000*"j" + 0.000*"t" + 0.000*"mammy" + 0.000*"new" + 0.000*"man" + 0.000*"house" + 0.000*"san" + 0.000*"s" + 0.000*"day" + 0.000*"bell"'),
 (3,
  '0.010*"day" + 0.008*"say" + 0.008*"street" + 0.008*"main" + 0.007*"store" + 0.007*"centro" + 0.007*"w" + 0.007*"el" + 0.006*"man" + 0.006*"today"'),
 (4,
  '0.042*"mammy" + 0.025*"bell" + 0.015*"house" + 0.014*"thomas" + 0.011*"year" + 0.009*"francisco" + 0.009*"san" + 0.009*"death" + 0.009*"power" + 0.008*"time"'),
 (5,
  '0.012*"j" + 0.008*"m" + 0.008*"t" + 0.008*"b" + 0.007*"l" + 0.006*"day" + 0.006*"n" + 0.006*"city" + 0.005*"club" + 0.005*"r"'),
 (6,
  '0.000*"man" + 0.000*"california" + 0.000*"day" + 0.000

In [30]:
# Find topics in each document
lda_model.get_document_topics(corpus[0])

[(0, 0.99937576)]

In [33]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
19     0.106283+0.000000j  0.046436+0.000000j       1        1  19.554790
5      0.100978+0.000000j  0.030933+0.000000j       2        1  17.926765
10     0.059208+0.000000j  0.063614+0.000000j       3        1  15.941666
0     -0.150098+0.000000j  0.102909+0.000000j       4        1  10.896518
4     -0.176235+0.000000j  0.138408+0.000000j       5        1   9.526787
13     0.123773+0.000000j  0.111904+0.000000j       6        1   9.147615
3      0.059088+0.000000j  0.116280+0.000000j       7        1   8.760931
9     -0.164891+0.000000j -0.055964+0.000000j       8        1   8.201952
7      0.003491+0.000000j -0.046210+0.000000j       9        1   0.003581
8      0.003490+0.000000j -0.046208+0.000000j      10        1   0.003581
1      0.003491+0.000000j -0.046210+0.000000j      11        1   0.003581
6      

### Tf-idf topic model

In [ ]:
# initialize a tfidf model
tfidf = models.TfidfModel(corpus)

In [ ]:
# make a new corpus based on the tfidf model
corpus_tfidf = tfidf[corpus]

In [ ]:
# here we build our topic model
lda_model_tfidf = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes=50)
corpus_lda = lda_model_tfidf[corpus_tfidf]

In [ ]:
lda_model_tfidf.print_topics()

In [ ]:
# Find topics in each document
lda_model_tfidf.get_document_topics(corpus_tfidf[0])

In [ ]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis